In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "klyukinv"
os.environ['KAGGLE_KEY'] = "6fc43bd58892df21aa88d9f8ad6206ef"

In [4]:
!pip install --user kaggle

  Using cached https://files.pythonhosted.org/packages/a6/a5/c0b6468d3824fe3fde30dbb5e1f687b291608f9473681bbf7dabbf5a87d7/text_unidecode-1.3-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
!kaggle competitions download -c 2018-hse-ml-competition-04
!unzip 2018-hse-ml-competition-04.zip

2018-hse-ml-competition-04.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2018-hse-ml-competition-04.zip
  inflating: items.json.gz           
  inflating: random_benchmark.csv    
  inflating: train.json.gz           


In [7]:
!pip install --user lightfm

You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
import gzip
import shutil

path = '../data/raw/'
filenames = ['items.json.gz', 'train.json.gz']
for filename_arch in filenames:
    with gzip.open(path + filename_arch, 'rb') as f_in:
        filename_new = path + '.'.join(filename_arch.split('.')[:-1])
        with open(filename_new, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [12]:
!pip install --user pymorphy2

     |████████████████████████████████| 51kB 716kB/s eta 0:00:011
     |████████████████████████████████| 7.1MB 3.2MB/s eta 0:00:01     |▏                               | 30kB 6.4MB/s eta 0:00:02
  Stored in directory: /home/valeriy/.cache/pip/wheels/9b/04/dd/7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import scipy.sparse
import json
import string
import pymorphy2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm_notebook
from multiprocessing import Pool, cpu_count

In [6]:
chars_to_replace = string.punctuation + '«»\n--––'
table = str.maketrans(chars_to_replace, ' ' * len(chars_to_replace))
morph = pymorphy2.MorphAnalyzer()

def clean_text(s):
    return " ".join(
        [morph.normal_forms(word)[0] for word in s.translate(table).lower().split()]
    )

def process_line(line):
    item = json.loads(line)
    if isinstance(item['image'], float):
        item['image'] = np.full((96,),0)
    else:
        item['image'] = np.array(item['image'])
    item['content'] = clean_text(item['content'])
    item['title'] = clean_text(item['title'])
    return item

In [4]:
items_count = 0
with open('../data/interim/items.json') as items_file:
    for line in items_file:
        items_count += 1
items_count

328050

In [7]:
items = []
with open('../data/interim/items.json') as items_file:
    with Pool(cpu_count()) as p:
        items = list(tqdm_notebook(p.imap(process_line, items_file), total=items_count))
items = pd.DataFrame(items)
items.head()

,content,image,itemId,title
0,согласиться дорогой любитель собака до что же ...,"[-0.169, 0.129, 0.067, 0.019, 0.281, -0.245, 0...",0,пять забавный морщинистый порода собака
1,контур три поперечный улица состоять до недавн...,"[-0.158, -0.112, -0.325, 0.05, -0.114, 0.002, ...",1,история улица ирининский в гомель
2,источник http infodays ru вообще он как то сам...,"[0.084, -0.181, 0.008, 0.34, -0.03, -0.197, -0...",2,зачем дудь весь время спрашивать гость програм...
3,41 летний светлана зейналов решить окрестить 5...,"[0.034, -0.119, -0.062, 0.025, 0.128, -0.041, ...",3,светлана зейналов крестить младший дочь
4,организовать преступный группировка гбао делат...,"[-0.061, -0.015, -0.198, -0.047, 0.054, 0.029,...",4,гкнб бандит в гбао делать вид что расстаться с...


In [9]:
items.to_csv('../data/processed/processed_items.csv')

In [22]:
data = []
row = []
col = []

train_lines = sum(1 for line in open('train.json','r'))

with open('train.json') as train_file:
    for i, line in enumerate(tqdm_notebook(train_file, total=train_lines)):
        json_line = json.loads(line)
        for item, rating in json_line['trainRatings'].items():
            data.append((-1) ** (int(rating) + 1))
            row.append(i)
            col.append(int(item))
train_int = scipy.sparse.coo_matrix((data, (row, col)))

In [24]:
del data, row, col

In [25]:
identity_items = scipy.sparse.eye(len(items))

In [26]:
import lightfm

In [27]:
model = lightfm.LightFM(no_components=128, loss='logistic', random_state=42)
model.fit(train_int, epochs=7, num_threads=cpu_count(), item_features=identity_items)

In [28]:
sample = pd.read_csv('predictions/random_benchmark.csv')
sample['pred'] = model.predict(
    sample.userId.values,
    sample.itemId.values,
    item_features=identity_items,
    num_threads=cpu_count(),
)
sample.sort_values(['userId', 'pred'], ascending=[True, False], inplace=True)
sample.drop(columns=['pred'], inplace=True)
sample.to_csv('predictions/lightfm_identity_items.csv', index=False)

In [29]:
!kaggle competitions submit -c 2018-hse-ml-competition-04 -f predictions/lightfm_identity_items.csv -m "Raw identity items submission"

100%|██████████████████████████████████████| 35.7M/35.7M [00:06<00:00, 5.36MB/s]
Successfully submitted to Рекомендательная система для статей